In [46]:
import torch
# origin_pth = torch.load('../checkpoints/effvit/eff_b1.pth' ,map_location=torch.device('cuda'))
origin_pth = torch.load('../checkpoints/b3-r1184.pt' ,map_location=torch.device('cuda'))
# origin_pth = torch.load('../pose1.pth' ,map_location=torch.device('cuda'))

In [47]:
new_model_state = origin_pth
for k in list(new_model_state["state_dict"]):
    if "stages.0.op_list" not in k and "stages.1.op_list" not in k:
        if "op_list" in k and ("stages" in k or "input_stem" in k):
            print(k)
            ori = k
            new_k = ori.replace("op_list.", "")
            new_model_state["state_dict"][new_k] = origin_pth["state_dict"][ori]
            del new_model_state["state_dict"][ori]
        elif "decode_head" in k:
            print(k)
            ori = k
            new_k = ori.replace("decode_head.", "head.")
            new_model_state["state_dict"][new_k] = origin_pth["state_dict"][ori]
            del new_model_state["state_dict"][ori]

backbone.input_stem.op_list.0.conv.weight
backbone.input_stem.op_list.0.norm.weight
backbone.input_stem.op_list.0.norm.bias
backbone.input_stem.op_list.0.norm.running_mean
backbone.input_stem.op_list.0.norm.running_var
backbone.input_stem.op_list.0.norm.num_batches_tracked
backbone.input_stem.op_list.1.main.depth_conv.conv.weight
backbone.input_stem.op_list.1.main.depth_conv.norm.weight
backbone.input_stem.op_list.1.main.depth_conv.norm.bias
backbone.input_stem.op_list.1.main.depth_conv.norm.running_mean
backbone.input_stem.op_list.1.main.depth_conv.norm.running_var
backbone.input_stem.op_list.1.main.depth_conv.norm.num_batches_tracked
backbone.input_stem.op_list.1.main.point_conv.conv.weight
backbone.input_stem.op_list.1.main.point_conv.norm.weight
backbone.input_stem.op_list.1.main.point_conv.norm.bias
backbone.input_stem.op_list.1.main.point_conv.norm.running_mean
backbone.input_stem.op_list.1.main.point_conv.norm.running_var
backbone.input_stem.op_list.1.main.point_conv.norm.num_ba

In [48]:
import torch
import torch.nn.functional as F
from typing import Any, List, Optional, Tuple, Union, Dict, Callable, Type
# torch.set_printoptions(threshold=torch.inf)
def conweight(net, key2, out_c, in_c, head, dim, k, offset):
    res = torch.tensor(torch.zeros(in_c, in_c, k, k), device='cuda')
    for i in range(head):
        res[dim*i:dim*(i+1)] = net['state_dict'][key2][offset+dim*3*i:offset+dim*3*i+dim].clone()
        print(f"res[{dim*i}:{dim*(i+1)}]=net['state_dict']['{key2}'][{offset+dim*3*i}:{offset+dim*3*i+dim}]")
    return res

def conweight1(net, key2, out_c, in_c, head, dim, k, offset):
    res = torch.tensor(torch.zeros(in_c, 1, k, k), device='cuda')
    for i in range(head):
        res[dim*i:dim*(i+1)] = net['state_dict'][key2][offset+dim*3*i:offset+dim*3*i+dim].clone()
        print(f"res[{dim*i}:{dim*(i+1)}]=net['state_dict']['{key2}'][{offset+dim*3*i}:{offset+dim*3*i+dim}]")
    return res

def conweight2(net, key2, out_c, in_c, head, dim, k, offset):
    res = torch.tensor(torch.zeros(in_c, dim, k, k), device='cuda')
    for i in range(head):
        res[dim*i:dim*(i+1)] = net['state_dict'][key2][offset+dim*3*i:offset+dim*3*i+dim].clone()
        print(f"res[{dim*i}:{dim*(i+1)}]=net['state_dict']['{key2}'][{offset+dim*3*i}:{offset+dim*3*i+dim}]")
    return res

def get_same_padding(kernel_size: Union[int, Tuple[int, ...]]) -> Union[int, Tuple[int, ...]]:
    if isinstance(kernel_size, tuple):
        return tuple([get_same_padding(ks) for ks in kernel_size])
    else:
        assert kernel_size % 2 > 0, "kernel size should be odd number"
        return kernel_size // 2

def has_close_enough_values(x, y, threshold=0.000001, percentage=0.999999):
    rel_err = torch.abs(x - y) / torch.abs(y)
    close_count = torch.sum(rel_err < threshold).item()
    total_count = x.numel()
    return close_count / total_count >= percentage
        

def all_in(net, a, b, out_c, in_c, head, dim, k):
    res_q_1 = conweight(net, f'backbone.stages.{a}.{b}.context_module.main.qkv.conv.weight',out_c,in_c,head,dim,1,0)
    res_k_1 = conweight(net, f'backbone.stages.{a}.{b}.context_module.main.qkv.conv.weight',out_c,in_c,head,dim,1,dim*1)
    res_v_1 = conweight(net, f'backbone.stages.{a}.{b}.context_module.main.qkv.conv.weight',out_c,in_c,head,dim,1,dim*2)
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.qkv_layers.q.conv.weight'] = res_q_1
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.qkv_layers.k.conv.weight'] = res_k_1
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.qkv_layers.v.conv.weight'] = res_v_1
    del net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.qkv.conv.weight'] 
    
    res_q_2 = conweight1(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.0.weight',out_c,in_c,head,dim,5,0)
    res_k_2 = conweight1(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.0.weight',out_c,in_c,head,dim,5,dim*1)
    res_v_2 = conweight1(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.0.weight',out_c,in_c,head,dim,5,dim*2)
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.q.0.0.weight'] = res_q_2
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.k.0.0.weight'] = res_k_2
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.v.0.0.weight'] = res_v_2
    del net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.0.weight'] 

    res_q_3 = conweight2(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.1.weight',out_c,in_c,head,dim,1,0)
    res_k_3 = conweight2(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.1.weight',out_c,in_c,head,dim,1,dim*1)
    res_v_3 = conweight2(net, f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.1.weight',out_c,in_c,head,dim,1,dim*2)
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.q.0.1.weight'] = res_q_3
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.k.0.1.weight'] = res_k_3
    net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.v.0.1.weight'] = res_v_3
    del net['state_dict'][f'backbone.stages.{a}.{b}.context_module.main.aggreg.0.1.weight'] 
    

In [49]:
net = origin_pth
for key, value in list(net['state_dict'].items()):
    if 'qkv' in key or 'aggre' in key: print(key, value.size(), sep=" ")

backbone.stages.2.1.context_module.main.qkv.conv.weight torch.Size([768, 256, 1, 1])
backbone.stages.2.1.context_module.main.aggreg.0.0.weight torch.Size([768, 1, 5, 5])
backbone.stages.2.1.context_module.main.aggreg.0.1.weight torch.Size([768, 32, 1, 1])
backbone.stages.2.2.context_module.main.qkv.conv.weight torch.Size([768, 256, 1, 1])
backbone.stages.2.2.context_module.main.aggreg.0.0.weight torch.Size([768, 1, 5, 5])
backbone.stages.2.2.context_module.main.aggreg.0.1.weight torch.Size([768, 32, 1, 1])
backbone.stages.2.3.context_module.main.qkv.conv.weight torch.Size([768, 256, 1, 1])
backbone.stages.2.3.context_module.main.aggreg.0.0.weight torch.Size([768, 1, 5, 5])
backbone.stages.2.3.context_module.main.aggreg.0.1.weight torch.Size([768, 32, 1, 1])
backbone.stages.2.4.context_module.main.qkv.conv.weight torch.Size([768, 256, 1, 1])
backbone.stages.2.4.context_module.main.aggreg.0.0.weight torch.Size([768, 1, 5, 5])
backbone.stages.2.4.context_module.main.aggreg.0.1.weight torc

In [50]:
# b0
# all_in(net,2,1,192,64,4,16,5)
# all_in(net,2,2,192,64,4,16,5)
# all_in(net,3,1,384,128,8,16,5)
# all_in(net,3,2,384,128,8,16,5)
# b1
# all_in(net,2,1,384,128,8,16,5)
# all_in(net,2,2,384,128,8,16,5)
# all_in(net,2,3,384,128,8,16,5)
# all_in(net,3,1,768,256,16,16,5)
# all_in(net,3,2,768,256,16,16,5)
# all_in(net,3,3,768,256,16,16,5)
# all_in(net,3,4,768,256,16,16,5)
# b3
all_in(net,2,1,768,256,8,32,5)
all_in(net,2,2,768,256,8,32,5)
all_in(net,2,3,768,256,8,32,5)
all_in(net,2,4,768,256,8,32,5)
all_in(net,2,5,768,256,8,32,5)
all_in(net,2,6,768,256,8,32,5)

all_in(net,3,1,1536,512,16,32,5)
all_in(net,3,2,1536,512,16,32,5)
all_in(net,3,3,1536,512,16,32,5)
all_in(net,3,4,1536,512,16,32,5)
all_in(net,3,5,1536,512,16,32,5)
all_in(net,3,6,1536,512,16,32,5)
all_in(net,3,7,1536,512,16,32,5)
all_in(net,3,8,1536,512,16,32,5)
all_in(net,3,9,1536,512,16,32,5)

new_state_dict = net
# for k, v in net.items():
#     new_state_dict[k] = v
    
        
for k in list(new_state_dict["state_dict"]):
    if "stages.0.op_list" not in k and "stages.1.op_list" not in k:
        if "op_list" in k and ("stages" in k or "input_stem" in k):
            print(k)
            ori = k
            new_k = ori.replace("op_list.", "")
            new_state_dict["state_dict"][new_k] = origin_pth["state_dict"][ori]
            del new_state_dict["state_dict"][ori]
        elif "decode_head" in k:
            print(k)
            ori = k
            new_k = ori.replace("decode_head.", "head.")
            new_state_dict["state_dict"][new_k] = origin_pth["state_dict"][ori]
            del new_state_dict["state_dict"][ori]

res[0:32]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][0:32]
res[32:64]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][96:128]
res[64:96]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][192:224]
res[96:128]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][288:320]
res[128:160]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][384:416]
res[160:192]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][480:512]
res[192:224]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][576:608]
res[224:256]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][672:704]
res[0:32]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][32:64]
res[32:64]=net['state_dict']['backbone.stages.2.1.context_module.main.qkv.conv.weight'][128:160]
res[64:96]=net['state_dict'][

/tmp/ipykernel_3813222/2174001680.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = torch.tensor(torch.zeros(in_c, in_c, k, k), device='cuda')
/tmp/ipykernel_3813222/2174001680.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = torch.tensor(torch.zeros(in_c, 1, k, k), device='cuda')
/tmp/ipykernel_3813222/2174001680.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = torch.tensor(torch.zeros(in_c, dim, k, k), device='cuda')


res[128:160]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][416:448]
res[160:192]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][512:544]
res[192:224]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][608:640]
res[224:256]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][704:736]
res[256:288]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][800:832]
res[288:320]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][896:928]
res[320:352]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][992:1024]
res[352:384]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][1088:1120]
res[384:416]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][1184:1216]
res[416:448]=net['state_dict']['backbone.stages.3.3.context_module.main.qkv.conv.weight'][1280:1312]
res

res[0:32]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][0:32]
res[32:64]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][96:128]
res[64:96]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][192:224]
res[96:128]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][288:320]
res[128:160]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][384:416]
res[160:192]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][480:512]
res[192:224]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][576:608]
res[224:256]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][672:704]
res[256:288]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][768:800]
res[288:320]=net['state_dict']['backbone.stages.3.8.context_module.main.qkv.conv.weight'][864:896]
res[320:352]=net['stat

In [51]:
torch.save(net, '../checkpoints/eff_b3_branch.pth')
# torch.save(net, '../pose_new.pth')
# torch.save(new_state_dict, '../checkpoints/effvit/eff_b1_branch.pth')